<a href="https://colab.research.google.com/github/Slaxzer96/klamkode/blob/master/FPSULSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.nn import functional as F
from tqdm.notebook import tqdm





In [3]:
class AutoUpscale(nn.Module):
    def __init__(self, input_size=6):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.BatchNorm2d(input_size),
            nn.Conv2d(input_size,32,5,1,2),    # 32
            nn.ReLU(),        
            nn.BatchNorm2d(32),
        
            nn.Conv2d(32, 64, 3),              # 30
            nn.BatchNorm2d(32),
            nn.ReLU(),
        
            nn.Conv2d(32, 64, 3),              # 28
            nn.ReLU())
            
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 3),     # 30
            nn.BatchNorm2d(32),
            nn.ReLU(),

            nn.ConvTranspose2d(32, 16, 3),     # 32
            nn.BatchNorm2d(16),
            nn.ReLU(),

            nn.Conv2d(16, 8, 3, 1, 1),     # 32
            nn.BatchNorm2d(8),
            nn.ReLU(),

            nn.Conv2d(8, 3, input_size//2, 1, 1),     # 32
            nn.ReLU())

    def decode(self, x: torch.Tensor):
        return self.decode(x)

    def encode(self, x: torch.Tensor):
        return self.encode(x)

    def forward(self, x: torch.Tensor):
        return self.decode(self.encode(x))

    def __repr__(self):
        return "AutoUpscale_siamese"
        





In [ ]:
class FPSULSTM(nn.Module):
    def __init__(self, d=512, n_lstm_layers=1, dropout=0.):
        super().__init__()
        self.emb = nn.Embedding(128, d)  # max(emails_ascii) < 128
        self.lstm = nn.LSTM(d, d, n_lstm_layers, batch_first=True, dropout=dropout)
        self.lin = nn.Linear(d, 1)

    def forward(self, x, end_idx):  # (B, nx)
        x = self.emb(x)  # (B, nx, d)
        y = self.lstm(x)[0][torch.arange(len(x)), end_idx]  # (B, d)
        return self.lin(y).view(-1)  # (B,)


In [ ]:
class ConvLSTM(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.ii = nn.Conv2d(input_size, input_size, 5, 1, 2)
        self.ih = nn.Conv2d(input_size, input_size, 5, 1, 2)
        self.ig = nn.

In [ ]:
all_losses = {}
best_model = None


In [ ]:

autoUpscale = AutoUpscale(6).cuda()

models = [autoUpscale]

for model in models:
    model.cuda()

     opt = torch.optim.Adam(model.parameters(), lr=alpha)
        train_losses = []
        best_loss = float("inf")

        for epoch in tqdm(range(15)):
            # train
            epoch_losses = []

            model.train()
            for x, y in loader_train:
                x = x.cuda()
                reconstruction = model(x)
                loss = F.l1_loss(reconstruction, y)

                opt.zero_grad()
                loss.backward()
                opt.step()

                mae = F.l1_loss(reconstruction, x)

                epoch_losses.append(loss.item())

            train_loss = np.mean(epoch_losses)
            
            print(f' rmse: {np.sqrt(loss.item()):.4f} - mae: {mae.item():.4f}')

            #if train_loss < best_loss:
            #    best_loss = train_loss
            #best_model = copy.deepcopy(model), epoch

            train_losses.append(train_loss)


        all_losses[alpha, model] = train_losses

model.eval()
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.xlabel('epoch')
plt.ylabel('loss')
for n, (train_losses) in all_losses.items():
  p = plt.plot(train_losses, label=f'{n}:train')
plt.legend()
plt.grid()

